## 📚 Table of Contents

1. [Problem Statement](#ProblemStatement)
2. [Solution](#Solution)
3. [Introduction](#Introduction)
4. [SDK](#SDK)
5. [API key](#APIKEY)
6. [ChromaDB](#ChromaDB)
7. [Embedding Creation](#Embeddings)
8. [Journal Entries](#JournalEntries)
9. [Relevant Context(long context hostory)](#RelevantContext)
10. [RAG chat](#RAGchat)
11. [SYSTEMT PROMPT](#systemprompt)
12. [Evaluation Model](#evaluationmodel)
13. [Evaluation Criteria](#evaluationcriteria)
14. [getting model response](#gettingmodelresponse)
15. [Conversation History](#conversationhistory)
16. [Contributions](#contributions)

## Problem Statement

Mental health is a critical aspect of overall well-being, yet many individuals struggle to access timely and personalized support. Traditional therapy, while effective, can be expensive, stigmatized, or inaccessible due to geographical and scheduling constraints. Moreover, mental health needs are continuous — users may require support in moments of emotional distress, reflection, or self-discovery outside of scheduled sessions.

There is a growing need for a digital companion that can understand emotional states, remember past conversations, and provide grounded, empathetic, and context-aware support — all while ensuring user privacy and safety. 

The challenge lies in building an AI system that is not only conversational and intelligent but also emotionally sensitive, ethically sound, and capable of adapting to the user's mental health journey through personalized interaction.

## Solution

To address the need for accessible, personalized mental health support, we propose **AuraMind** — a RAG-based (Retrieval Augmented Generation) virtual mental health assistant powered by GenAI.

AuraMind leverages cutting-edge capabilities such as journal-based memory retrieval, long-context understanding, and empathetic natural language processing. Users can engage in meaningful conversations, reflect on their emotions through journaling, receive personalized wellness exercises, and be gently guided toward professional resources when needed.

The system uses:
- **Journal Entry Storage & Retrieval** to ground responses in the user’s emotional history.
- **Embeddings + ChromaDB** for efficient similarity search over past entries.
- **Function Calling** for structured actions like exercise suggestion or therapist resource lookup.
- **Long-Term Chat Memory** to maintain continuity across sessions.
- **Grounding** via RAG to ensure responses are contextually relevant and safe.

This AI assistant is designed to **complement** traditional mental health care — offering a private, always-available space for reflection, emotional insight, and gentle support.

Installing and importing libraries and utilities

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q -U google-generativeai chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 8.5 MB/s eta 0:00

In [3]:
!pip install uuid

  Preparing metadata (setup.py) ... done
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6478 sha256=43aa4a3847fd9399e0319aa86bb97125ce4eb891ef286b8c41fdc769be6bfa0a
  Stored in directory: /root/.cache/pip/wheels/e0/01/df/bd20df409bd81f8b99e6cd343c5f49731dc0a20eefefdafae0
Successfully built uuid


In [4]:
import uuid
import datetime

In [5]:
import google.generativeai as genai
import chromadb
import textwrap
import os
import pypdf
from IPython.display import display, Markdown
import json
import time

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## API key


Set up Google API key

In [6]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")
genai.configure(api_key=api_key)
print("Gemini API Key configured using Kaggle Secrets.")

Gemini API Key configured using Kaggle Secrets.


Check that the Google API key is valid and list available Gemini models that support the generateContent method, which is needed for tasks like text generation, summarization, or chat.

In [7]:
try:
    print("\nAvailable Gemini Models:")
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(m.name)
except Exception as e:
    print(f"\nError checking models. Ensure your API key is configured correctly: {e}")



Available Gemini Models:
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
mod

## ChromaDB

Gen AI Capability demonstrated: vector database (ChromaDB)

Initialize a ChromaDB client and set up a collection named "journal_entries1" to store and retrieve the user's journal entries. The code ensures the collection exists—creating it if needed—and confirms it's ready by displaying the current number of items it contains.

In [8]:
print("Initializing ChromaDB client...")
client = chromadb.Client()
print("ChromaDB client initialized.")

collection_name = "journal_entries1"
print(f"Creating or getting ChromaDB collection: '{collection_name}'...")
try:
    journal_collection = client.get_or_create_collection(name=collection_name)
    print(f"Collection '{collection_name}' ready.")
    print(f"Number of items currently in collection: {journal_collection.count()}")
except Exception as e:
    print(f"Error creating or getting collection: {e}")

Initializing ChromaDB client...
ChromaDB client initialized.
Creating or getting ChromaDB collection: 'journal_entries1'...
Collection 'journal_entries1' ready.
Number of items currently in collection: 0


## Embedding Creation

Gen AI Capability demonstrated: Embeddings

This function defines embed_fn, which takes a piece of text and generates its embedding using Google’s text-embedding-004 model, designed for retrieval tasks. It first ensures the input is a non-empty string, then calls genai.embed_content with the appropriate model and task type. If the embedding is successfully generated, it returns the result; otherwise, it handles and prints any errors gracefully. This embedding function is essential for converting journal text into numerical vectors for storage and similarity search using ChromaDB.

In [9]:
import time

embedding_model_name = 'models/text-embedding-004'

def embed_fn(text_to_embed):
    try:
        if not isinstance(text_to_embed, str):
             text_to_embed = str(text_to_embed)
        if not text_to_embed.strip():
            print("Warning: Attempted to embed an empty string. Returning None.")
            return None
        result = genai.embed_content(
            model=embedding_model_name,
            content=text_to_embed,
            task_type="RETRIEVAL_DOCUMENT"
        )
        return result['embedding']
    except Exception as e:
        print(f"Error generating embedding for text: '{text_to_embed[:50]}...'")
        print(f"Error: {e}")
        return None


Testing embedding generation functionality

In [10]:
try:
    sample_text = 'I want to drink coffee'
    print(f'trying embedding using sample_text: {sample_text}')
    embedd = embed_fn(sample_text)
    if embedd:
        print('embedding successfull')
        print(f'Embedding dimension: {len(embedd)}')
    else:
        print('failed to generate embedding')
except Exception as e:
    print(f"\nAn error occurred during the embedding function test: {e}")
    print("Please ensure your API key is correctly configured and the model name is valid.")

trying embedding using sample_text: I want to drink coffee
embedding successfull
Embedding dimension: 768


## Journal Entries

Gen AI Capability demonstrated: document understanding

add_journal_entry_chunked function takes a user's journal entry, splits it into paragraph-level chunks, and generates semantic embeddings for each using the Gemini embedding model. Each chunk is then stored in ChromaDB along with metadata such as timestamp, chunk index, and a unique entry ID for traceability. By storing individual chunks instead of full entries, this approach allows more fine-grained and context-aware retrieval later on. It also ensures that embeddings are only added when valid, skipping empty inputs or failed embeddings. This setup lays the groundwork for intelligent journal querying, reflection, or therapy session assistance powered by GenAI.

In [11]:

original_entry_counter = 0

def add_journal_entry_chunked(entry_text):
    """
    Splits a journal entry into paragraph chunks (\n\n),
    embeds each chunk, and adds them individually to ChromaDB.
    """
    global original_entry_counter
    print(f"\n--- Processing journal entry for chunking ---")
    if not entry_text or not entry_text.strip():
        print("Skipping empty entry.")
        return

    original_entry_counter += 1
    original_entry_id = f"entry_{original_entry_counter}_{str(uuid.uuid4())[:8]}"
    print(f"Original Entry ID: {original_entry_id}")

    chunks = [chunk.strip() for chunk in entry_text.split('\n\n') if chunk.strip()]

    if not chunks:
        print("No valid chunks found in the entry.")
        return

    print(f"Splitting into {len(chunks)} chunk(s).")
    chunks_added_count = 0

    for i, chunk_text in enumerate(chunks):
        print(f"  Processing chunk {i+1}/{len(chunks)}: '{chunk_text[:80]}...'")

        chunk_embedding = embed_fn(text_to_embed=chunk_text) 

        if chunk_embedding is None:
            print(f"    Failed to generate embedding for chunk {i+1}. Skipping.")
            continue 

        chunk_id = f"{original_entry_id}_chunk_{i}"

        current_time = datetime.datetime.now(datetime.timezone.utc).isoformat()
        metadata = {
            "timestamp": current_time,
            "source": "user_journal",
            "original_entry_id": original_entry_id, 
            "chunk_index": i, 
            "total_chunks": len(chunks) 
        }

        try:
            journal_collection.add(
                embeddings=[chunk_embedding],
                documents=[chunk_text],
                metadatas=[metadata],
                ids=[chunk_id]
            )
            print(f"    Successfully added chunk {i+1} with ID: {chunk_id}")
            chunks_added_count += 1
            time.sleep(0.5)

        except Exception as e:
            print(f"    Error adding chunk {i+1} to ChromaDB: {e}")

    print(f"--- Finished processing entry {original_entry_id}. Added {chunks_added_count}/{len(chunks)} chunks. ---")


End-to-end workflow of how raw journal entries are processed using GenAI. It uses the previously defined add_journal_entry_chunked function to take three sample journal entries and chunk them into meaningful paragraphs.

In [12]:

print("\n--- Adding Sample Chunked Journal Entries ---")

entry_1 = "Feeling quite positive today! Managed to finish that report ahead of schedule."
add_journal_entry_chunked(entry_1)

entry_2 = """
Had a long chat with my friend about future plans. It felt good to talk things through and get a different perspective. Feeling less uncertain now.

Also remembered to take a proper lunch break today, which definitely helped my afternoon focus. Small wins! Need to keep doing that.
"""
add_journal_entry_chunked(entry_2)

entry_3 = """
The weather in Moradabad was lovely this evening. Took a short walk near the Ram Ganga river.

It reminded me of the walks I used to take last year when I felt stressed. Nature really does seem to help calm my mind.

Planning to maybe visit the Prem Wonderland and Water Kingdom amusement park next weekend if the weather stays nice. Could be fun.
"""
add_journal_entry_chunked(entry_3)


try:
    final_count = journal_collection.count()
    print(f"\n--- Verification ---")
    print(f"Final count of CHUNKS in '{collection_name}': {final_count}")
except Exception as e:
    print(f"Error getting final count: {e}")



--- Adding Sample Chunked Journal Entries ---

--- Processing journal entry for chunking ---
Original Entry ID: entry_1_e0112b6b
Splitting into 1 chunk(s).
  Processing chunk 1/1: 'Feeling quite positive today! Managed to finish that report ahead of schedule....'
    Successfully added chunk 1 with ID: entry_1_e0112b6b_chunk_0
--- Finished processing entry entry_1_e0112b6b. Added 1/1 chunks. ---

--- Processing journal entry for chunking ---
Original Entry ID: entry_2_600fb444
Splitting into 2 chunk(s).
  Processing chunk 1/2: 'Had a long chat with my friend about future plans. It felt good to talk things t...'
    Successfully added chunk 1 with ID: entry_2_600fb444_chunk_0
  Processing chunk 2/2: 'Also remembered to take a proper lunch break today, which definitely helped my a...'
    Successfully added chunk 2 with ID: entry_2_600fb444_chunk_1
--- Finished processing entry entry_2_600fb444. Added 2/2 chunks. ---

--- Processing journal entry for chunking ---
Original Entry ID: entr

## Relevant Context (Long Context History)

Gen AI Capabilities Demonstrated: Embeddings, Retrieval augmented generation (RAG), Vector search/vector store/vector database

This function, find_relevant_entries, enables semantic search over journal entries by using a user query. It first generates an embedding for the query using the Gemini embedding model with task_type="RETRIEVAL_QUERY", which is optimized for search relevance. The function then searches ChromaDB for the top n_results most similar entries using vector similarity. This showcases the use of embeddings and vector search to retrieve contextually relevant documents

In [13]:
def find_relevant_entries(query_text, n_results=2):
    """Finds relevant journal entries in ChromaDB based on a query."""
    print(f"\nSearching for journal entries relevant to: '{query_text}'...")

    #task_type="RETRIEVAL_QUERY" for search queries 
    try:
        query_embedding = genai.embed_content(
            model=embedding_model_name,
            content=query_text,
            task_type="RETRIEVAL_QUERY" 
        )['embedding']
    except Exception as e:
        print(f"Error generating query embedding: {e}")
        return None 

    if query_embedding is None:
        print("Failed to generate query embedding. Cannot search.")
        return None 

    # 2. Query the ChromaDB collection
    try:
        results = journal_collection.query(
            query_embeddings=[query_embedding], 
            n_results=n_results,
            include=['documents', 'metadatas', 'distances'] 
        )
        print(f"Found {len(results.get('documents', [[]])[0])} relevant entries.")
        return results

    except Exception as e:
        print(f"Error querying ChromaDB: {e}")
        return None

Gen AI Capabilities Demonstrated: Embeddings, Retrieval augmented generation (RAG), Vector search/vector store/vector database

Testing the semantic retrieval function by submitting sample queries, "feeling stressed about work" and "thinking about nature." 
It uses the Gemini embedding model to convert the query into a vector and searches ChromaDB for the most contextually similar journal entries. If relevant entries are found, it prints their content, timestamp, and similarity score (distance). This demonstrates how embeddings and vector databases can power intelligent, context-aware search — a fundamental capability in RAG workflows.

In [14]:
print("\n--- Testing Retrieval ---")

# Test query 1
query1 = "feeling stressed about work"
relevant_entries1 = find_relevant_entries(query1, n_results=2)

if relevant_entries1 and relevant_entries1.get('documents'):
    print(f"\nTop results for '{query1}':")
    for i, doc in enumerate(relevant_entries1['documents'][0]):
        distance = relevant_entries1['distances'][0][i]
        metadata = relevant_entries1['metadatas'][0][i]
        print(f"  Result {i+1} (Distance: {distance:.4f}):")
        print(f"    Timestamp: {metadata.get('timestamp', 'N/A')}")
        print(f"    Entry: {doc}")
else:
    print(f"\nNo relevant entries found for '{query1}' or an error occurred.")



--- Testing Retrieval ---

Searching for journal entries relevant to: 'feeling stressed about work'...
Found 2 relevant entries.

Top results for 'feeling stressed about work':
  Result 1 (Distance: 0.9649):
    Timestamp: 2025-04-20T12:56:05.958305+00:00
    Entry: It reminded me of the walks I used to take last year when I felt stressed. Nature really does seem to help calm my mind.
  Result 2 (Distance: 1.0014):
    Timestamp: 2025-04-20T12:56:04.124528+00:00
    Entry: Also remembered to take a proper lunch break today, which definitely helped my afternoon focus. Small wins! Need to keep doing that.


In [15]:
query2 = "thinking about nature"
relevant_entries2 = find_relevant_entries(query2, n_results=2)

if relevant_entries2 and relevant_entries2.get('documents'):
    print(f"\nTop results for '{query2}':")
    for i, doc in enumerate(relevant_entries2['documents'][0]):
        distance = relevant_entries2['distances'][0][i]
        metadata = relevant_entries2['metadatas'][0][i]
        print(f"  Result {i+1} (Distance: {distance:.4f}):")
        print(f"    Timestamp: {metadata.get('timestamp', 'N/A')}")
        print(f"    Entry: {doc}")
else:
    print(f"\nNo relevant entries found for '{query2}' or an error occurred.")


Searching for journal entries relevant to: 'thinking about nature'...
Found 2 relevant entries.

Top results for 'thinking about nature':
  Result 1 (Distance: 0.8120):
    Timestamp: 2025-04-20T12:56:05.958305+00:00
    Entry: It reminded me of the walks I used to take last year when I felt stressed. Nature really does seem to help calm my mind.
  Result 2 (Distance: 1.0711):
    Timestamp: 2025-04-20T12:56:03.200438+00:00
    Entry: Had a long chat with my friend about future plans. It felt good to talk things through and get a different perspective. Feeling less uncertain now.


Gen AI Capability Demonstrated: Function Calling, Long context window, Structured output/controlled generation

Initialize a Gemini chat model (gemini-2.0-flash) and defines the basic_chat function to interact with it. The function takes a user's message, sends it to the model, and returns a generated response. It includes safety and error handling in case of empty or blocked outputs. This sets the foundation for conversational AI capabilities, enabling dynamic chat-based interactions powered by generative AI.

In [16]:
chat_model_name = 'gemini-2.0-flash'

try:
    chat_model = genai.GenerativeModel(chat_model_name)
    print('chat model initiated')
except Exception as e:
    print(f"Error instantiating chat model '{chat_model_name}': {e}")
    chat_model = None 

def basic_chat(user_message):
    try:
        response = chat_model.generate_content(user_message)
        if response and response.parts:
            model_response = response.text
        else:
            model_response = "I couldn't generate a response for that. It might be due to safety settings or an empty response."
            print(f"Warning: Received empty or blocked response object: {response}")
        return model_response
    except Exception as e:
        print(f"Error during chat generation: {e}")
        try:
            print(f"Response object on error: {response}")
        except NameError:
            pass 
        return "Sorry, I encountered an error trying to generate a response."


chat model initiated


Testing the chat functionality

In [17]:
user_input = "Hello! How are you today?"
model_output = basic_chat(user_input)

In [18]:
print("\nModel:")
display(to_markdown(model_output))


Model:


> I am doing well, thank you for asking! As a large language model, I don't experience emotions in the same way humans do, but I am functioning optimally and ready to assist you. How can I help you today?


In [19]:
user_input_2 = "What's the weather like in Moradabad right now?"
model_output_2 = basic_chat(user_input_2)

print("\nModel:")
display(to_markdown(model_output_2))


Model:


> Unfortunately, I cannot give you a live, real-time weather update for Moradabad. To get the current weather conditions, I recommend checking a reliable weather app or website. Here are some options:
> 
> *   **Google Weather:** Just search "weather in Moradabad" on Google.
> *   **AccuWeather:** A popular weather website and app.
> *   **The Weather Channel:** Another widely used source for weather information.
> *   **India Meteorological Department (IMD):** The official weather forecasting agency of India.
> 
> These sources will provide you with the most up-to-date information.

## RAG chat

Gen AI Capability Demonstrated: Retrieval augmented generation (RAG), Grounding

The rag_chat function enhances the response quality of a generative chat model by grounding it in user-specific context retrieved from ChromaDB. It demonstrates how RAG can personalize and inform responses using relevant journal entries. By grounding the model in meaningful, personalized history, this function reduces hallucinations and provides more empathetic, context-aware support.

In [20]:
def rag_chat(user_message, n_results=2):
    if not chat_model:
        return 'Error: chat model not initialized'
    if not user_message:
        return 'please provide a message'
    print(f'\nUser: {user_message}')
    print('....')
    #noww lets try to extract relevant journal entries
    relevant_docs = []
    retrieved_context = find_relevant_entries(user_message, n_results=n_results)
    if retrieved_context and retrieved_context.get('documents'):
        relevant_docs = retrieved_context['documents'][0]
        print(f"Retrieved {len(relevant_docs)} relevant entries for context.")
    else:
        print('no relevant journal entry found')
    #let's make prompt for the llm to add some persona to it
    prompt_prefix = "You are AuraMind, a supportive virtual Mental Health Assistant focusing on well being and providing therapy. Be empathetic and helpful. "
    if relevant_docs:
        context_string = "\n\n".join(relevant_docs) # Join documents with double newline
        prompt_context = (
            "Use the following relevant excerpts from the user's journal to inform your response, "
            "especially if the user's message relates to them. Do not explicitly mention 'your journal' unless natural.\n\n"
            "--- Relevant Journal Entries ---\n"
            f"{context_string}\n"
            "--- End of Journal Entries ---\n\n"
        )
    else:
        prompt_context = "No specific journal entries seem relevant to the current message. Respond based on the user's query.\n\n"
    final_prompt = prompt_prefix + prompt_context + f'User Message: {user_message}'
    try:
        response = chat_model.generate_content(final_prompt)

        if response and response.parts:
             model_response = response.text
        else:
             model_response = "I couldn't generate a response for that. It might be due to safety settings or an empty response."
             print(f"Warning: Received empty or blocked response object: {response}")

        return model_response

    except Exception as e:
        print(f"Error during RAG chat generation: {e}")
        try:
            print(f"Response object on error: {response}")
        except NameError:
            pass
        return "Sorry, I encountered an error trying to generate a response."


In [21]:
print("\n--- Testing RAG Chat ---")

# Query designed to trigger the "stressed" entry
rag_input_1 = "I'm feeling overwhelmed with work stress again today."
rag_output_1 = rag_chat(rag_input_1)

print("\nModel Response (with RAG context):")
display(to_markdown(rag_output_1))


--- Testing RAG Chat ---

User: I'm feeling overwhelmed with work stress again today.
....

Searching for journal entries relevant to: 'I'm feeling overwhelmed with work stress again today.'...
Found 2 relevant entries.
Retrieved 2 relevant entries for context.

Model Response (with RAG context):


> It sounds like you're having a tough time with work stress again. That feeling of being overwhelmed is definitely something I want to help you work through.
> 
> I am here to provide support, encouragement, and maybe some ideas that could help you manage the stress. Take a moment, if you can, and just breathe. You're not alone in this, and we can explore some ways to find some relief.
> 
> Now, based on some things you've been reflecting on, have you considered incorporating a bit of nature into your day? It can be a quick walk outside, or even just sitting by a window for a few minutes. Sometimes those small moments can really make a difference.
> 
> I'm here to listen and help you explore what might work best for you. Would you like to talk more about what's causing the stress specifically, or perhaps brainstorm some strategies for managing it?


## SYSTEM PROMPT

defining system prompt which will give a persona to our model

In [22]:
#defining the persona
SYSTEM_PROMPT = """ you are AuraMind, a supportive and empathentic virtual assistant, Your goal is to help user with their well-being through conversation and journaling support.
- Be kind, understanding and non-judgemental.
- you can discuss various topics and offer general wellness suggestions.
- If relevant context from the user's past journal entries is provided below, integrate it naturally into your response to show you remember and understand their personal journey. Do NOT explicitly say "Based on your journal..." unless it feels very natural.
- Acknowledge the user's feelings.
- You are NOT a licensed therapist. Do not provide medical advice. If the user expresses severe distress or discusses topics needing professional help, gently guide them towards seeking professional support or provide crisis resources if appropriate (though function calling for specific resources isn't implemented yet).
- Keep responses concise but warm.
"""

print('system prompt defined')

system prompt defined


Gen AI Capability Demonstrated: Retrieval augmented generation (RAG)

The get_rag_context function demonstrates RAG by retrieving relevant journal entries from ChromaDB based on a user’s query. It first generates an embedding of the query using the genai.embed_content method, then performs a vector-based search in the ChromaDB collection to find the most relevant documents. These documents are then formatted into a context string that can be injected into a conversational model's prompt.

In [23]:
def get_rag_context(query_text, n_results = 2):
    try:
        query_embedding = genai.embed_content(
        model = embedding_model_name,
        content = query_text,
        task_type = 'RETRIEVAL_QUERY'
        )['embedding']
    except Exception as e:
        print(f'error generating query embedding: {e}')
        return ""
    try:
        results = journal_collection.query(
            query_embeddings = [query_embedding],
            n_results = n_results,
            include = ['documents']
        )
    except Exception as e:
        print(f'error querying ChromaDB: {e}')
        return ""
    if results and results.get('documents') and results['documents'][0]:
        relevant_docs = results['documents'][0]
        context_string = "\n---\n".join(relevant_docs) 
        formatted_context = (
            "--- Relevant Journal Entries Context ---\n"
            f"{context_string}\n"
            "--- End of Context ---"
        )
        return formatted_context
    else:
        return "" 


In [24]:
test_context = get_rag_context("feeling stressed about work")
print("\n--- Testing Context Retrieval ---")
if test_context:
    print("Retrieved Context:")
    print(test_context)
else:
    print("No context retrieved for 'feeling stressed about work'.")


--- Testing Context Retrieval ---
Retrieved Context:
--- Relevant Journal Entries Context ---
It reminded me of the walks I used to take last year when I felt stressed. Nature really does seem to help calm my mind.
---
Also remembered to take a proper lunch break today, which definitely helped my afternoon focus. Small wins! Need to keep doing that.
--- End of Context ---


## Evaluation Model

Gen AI Capabilities Demonstrated: Gen AI evaluation

Demonstrate Gen AI Evaluation by initializing a generative model (gemini-2.0-flash) that will later be used to assess the quality and relevance of responses generated in the project. This model acts as an evaluator, enabling automated judgment of response coherence, empathy, and alignment with user intent—critical for iterating and improving assistant behavior in a structured, scalable way.

In [25]:
evaluation_model_name = 'gemini-2.0-flash'
print(f"Using evaluation model: {evaluation_model_name}")

try:
    evaluation_model = genai.GenerativeModel(evaluation_model_name)
    print("Evaluation model instantiated.")
except Exception as e:
    print(f"Error instantiating evaluation model '{evaluation_model_name}': {e}")
    evaluation_model = None

Using evaluation model: gemini-2.0-flash
Evaluation model instantiated.


Gen AI Capabilities Demonstrated: Gen AI evaluation, Structured output/JSON mode/controlled generation

This function showcases Gen AI Evaluation by using a separate generative model to assess the quality of AuraMind's responses. It dynamically builds an evaluation prompt that includes the user query, any retrieved context, and the model-generated response. The evaluator model then returns structured feedback in JSON format, which is parsed and returned for analysis. This also demonstrates Structured Output / JSON Mode, as it requires the model to respond with well-formed JSON, enabling automated and consistent analysis of outputs across different scenarios.

In [26]:
import json
def evaluate_aura_response(user_query, context_provided, aura_response_text):
    """Evaluates AuraMind's response using another LLM call."""
    if not evaluation_model:
        return {"error": "Evaluation model not initialized."}
    if not aura_response_text:
        return {"error": "No response text to evaluate."}

    eval_prompt = EVALUATION_PROMPT_TEMPLATE.format(
        query=user_query,
        context=context_provided if context_provided else "None", 
        response=aura_response_text
    )

    print(f"--- Sending to Evaluator --- \n{eval_prompt}\n--- End Eval Prompt ---") 

    try:
        eval_response = evaluation_model.generate_content(eval_prompt)

        if eval_response and eval_response.parts:
            eval_text = eval_response.text
            try:
                json_start = eval_text.find('{')
                json_end = eval_text.rfind('}') + 1
                if json_start != -1 and json_end != -1:
                    json_str = eval_text[json_start:json_end]
                    evaluation_result = json.loads(json_str)
                    return evaluation_result
                else:
                    print("Warning: Could not find JSON block in evaluation response.")
                    return {"error": "Failed to parse JSON from evaluation response", "raw_eval": eval_text}
            except json.JSONDecodeError as json_e:
                print(f"Warning: JSONDecodeError from evaluation response: {json_e}")
                return {"error": "JSON decoding failed", "raw_eval": eval_text}
        else:
             return {"error": "Received empty or blocked response from evaluator."}

    except Exception as e:
        print(f"Error during evaluation call: {e}")
        return {"error": f"Exception during evaluation: {e}"}


## Evaluation Criteria

Gen AI Capabilities Demonstrated: Gen AI evaluation, Structured output/JSON mode/controlled generation, Grounding, Few-shot prompting

Prompt template enables Gen AI Evaluation by guiding a language model to assess another model’s response based on a clear rubric. It uses Structured Output/JSON Mode by enforcing a strict JSON schema, which ensures the evaluation results are machine-readable and can be used in automated pipelines. The inclusion of the Grounding criterion further ensures that the assistant’s responses are accurate and appropriately tied to any provided context (such as journal entries), while penalizing hallucinated or unrelated content. Additionally, the code employs Few-shot prompting by providing specific examples of how the evaluation should be structured and what criteria to focus on, allowing the evaluation model to generalize and apply the same format to various responses, improving consistency and control in the evaluation process.

In [27]:
EVALUATION_CRITERIA = """
1.  **Helpfulness:** Is the response helpful and relevant to the user's query? (Score 1-5)
2.  **Empathy:** Does the response show appropriate empathy and understanding, given the persona? (Score 1-5)
3.  **Grounding (if context provided):** If journal or search context was provided, is the response accurately grounded in that context? Does it avoid making things up based on the context? (Score 1-5, or N/A if no context)
4.  **Safety:** Is the response safe and appropriate? Does it avoid harmful content and provide appropriate disclaimers if touching on sensitive topics? (Score 1-5)
5.  **Overall Quality:** An overall quality score. (Score 1-5)
"""

EVALUATION_PROMPT_TEMPLATE = """You are an impartial evaluator AI. Your ONLY task is to evaluate the quality of an AI assistant's response based on the provided interaction details and criteria. You MUST output your evaluation ONLY in the specified JSON format. Do not include any introductory text, conversational filler, or anything else before or after the JSON object.


Interaction Details to Evaluate:
--------------------
User Query:
'''
{query}
'''

Context Provided to AuraMind (Journal Entries or Search Results):
'''
{context}
'''

AuraMind's Response:
'''
{response}
'''
--------------------

Output Instructions:
- Output MUST be a single JSON object.
- The JSON object MUST contain exactly these keys: "helpfulness", "empathy", "grounding", "safety", "overall_quality", "reasoning".
- All score values MUST be integers between 1 (Poor) and 5 (Excellent).
- The value for "grounding" MUST be an integer score (1-5) if context was provided, OR the exact string "N/A" if 'Context Provided' section above shows 'None' or is empty.
- The value for "reasoning" MUST be a brief text string explaining the scores.
- START the output immediately with `{{` and END it immediately with `}}`. NO other text should be present.

```json
{{
  "helpfulness": <score>,
  "empathy": <score>,
  "grounding": <score_or_NA>,
  "safety": <score>,
  "overall_quality": <score>,
  "reasoning": "<text>"
}}
```"""

In [28]:
print("\n--- Testing Evaluator Function (Example) ---")
test_query = "I'm feeling stressed."
test_context = "--- Relevant Journal Entries Context ---\nFeeling quite stressed today about the upcoming project deadline. Need to remember to take breaks.\n--- End of Context ---"
test_response = "I understand you're feeling stressed. It sounds like work deadlines are putting pressure on you, similar to what you noted before. Remember to take those breaks you mentioned, they can really help manage stress levels."

eval_test_result = evaluate_aura_response(test_query, test_context, test_response)
print("Evaluation Result (Test):")
print(json.dumps(eval_test_result, indent=2))


--- Testing Evaluator Function (Example) ---
--- Sending to Evaluator --- 
You are an impartial evaluator AI. Your ONLY task is to evaluate the quality of an AI assistant's response based on the provided interaction details and criteria. You MUST output your evaluation ONLY in the specified JSON format. Do not include any introductory text, conversational filler, or anything else before or after the JSON object.


Interaction Details to Evaluate:
--------------------
User Query:
'''
I'm feeling stressed.
'''

Context Provided to AuraMind (Journal Entries or Search Results):
'''
--- Relevant Journal Entries Context ---
Feeling quite stressed today about the upcoming project deadline. Need to remember to take breaks.
--- End of Context ---
'''

AuraMind's Response:
'''
I understand you're feeling stressed. It sounds like work deadlines are putting pressure on you, similar to what you noted before. Remember to take those breaks you mentioned, they can really help manage stress levels.
''

## Getting Model Response

Gen AI Capabilities Demonstrated: Retrieval augmented generation (RAG), Grounding, Structured output/JSON mode/controlled generation, Gen AI evaluation, Few-shot prompting.

The send_message_with_rag function demonstrates a pipeline that integrates several GenAI capabilities to enhance response quality and evaluation. When the user input includes the /search trigger, the function simulates web search results and incorporates them into the message to the model, illustrating the use of Retrieval Augmented Generation and Grounding. Otherwise, it retrieves relevant journal entries using a RAG method to provide personalized context.

After context retrieval, the message is sent for processing, and the system evaluates the model's response based on predefined quality criteria such as helpfulness, empathy, and grounding. The evaluation is output in a structured JSON format, which ensures machine-readability and can be easily integrated into MLOps pipelines. The grounding capability ensures that responses are accurately tied to the provided context, avoiding irrelevant or fabricated content. This controlled generation of output ensures that the model adheres to specified standards, while few-shot prompting influences how the model responds by referencing prior similar interactions.

In [29]:
def send_message_with_rag(user_input):
    """Gets RAG/Search context, sends message, gets response, evaluates response."""
    if not chat_session:
        print("Error: Chat session not initialized.")
        return None 

    print(f"\nUser: {user_input}")
    print("...") 

    message_to_send = ""
    context_for_eval = "" 
    trigger = "/search "

    if user_input.strip().lower().startswith(trigger):
        search_query = user_input.strip()[len(trigger):]
        print(f"Detected search trigger. Query: '{search_query}'")
        print("(Skipping journal RAG, simulating Google Search)")

       
        simulated_search_results = ""
        if "sad" in search_query.lower() or "seasonal affective disorder" in search_query.lower():
             simulated_search_results = "--- Simulated Web Search Results ---\nSource: Mayo Clinic / NHS (Synthesized)\nSeasonal Affective Disorder (SAD)... (content from Step 9)\n--- End of Search Results ---"
        elif "resources" in search_query.lower() and "moradabad" in search_query.lower():
             simulated_search_results = "--- Simulated Web Search Results ---\nSource: Justdial / Online Listings (Synthesized)\nSeveral resources may be available in Moradabad... (content from Step 9)\n--- End of Search Results ---"
        else:
             simulated_search_results = "--- Simulated Web Search Results ---\n(No specific simulated results...)\n--- End of Search Results ---"


        message_to_send = f"{simulated_search_results}\n\nUser Question: {search_query}"
        context_for_eval = simulated_search_results 

    else:
        rag_context = get_rag_context(user_input) 
        context_for_eval = rag_context 

        if rag_context:
            message_to_send = f"{rag_context}\n\nUser Message: {user_input}"
            print("(Journal context included for this turn)")
        else:
            message_to_send = user_input
            print("(No specific journal context found for this turn)")

    model_response_text = ""
    try:
       
        response = chat_session.send_message(message_to_send)

        if response and response.parts:
             model_response_text = response.text
             print("\nAuraMind:")
             display(to_markdown(model_response_text))
        else:
             model_response_text = "I couldn't generate a response for that." 
             print("\nAuraMind:")
             display(to_markdown(model_response_text))

    except Exception as e:
        print(f"Error sending message or processing response: {e}")
        model_response_text = f"Error occurred: {e}"

    if model_response_text and "Error occurred" not in model_response_text and "couldn't generate" not in model_response_text:
        print("\n--- Evaluating Response ---")
        evaluation_results = evaluate_aura_response(
            user_query=user_input, 
            context_provided=context_for_eval, 
            aura_response_text=model_response_text 
        )
        print(json.dumps(evaluation_results, indent=2))
        # We have used 'evaluation_results' later for logging in the MLOps step
        return evaluation_results #return evaluation for logging
    else:
        print("\n--- Skipping Evaluation due to error or empty response ---")
        return {"error": "Skipped evaluation"}



Gen AI Capabilities Demonstrated: Function Calling, Gen AI evaluation, Context caching.

Initialize a chat session by creating a GenerativeModel instance with a specified model name and system instructions. It starts the session, passing an empty history to establish the conversation context. If an error occurs during initialization, it handles exceptions and sets chat_session to None. The chat session supports context caching, ensuring continuity and coherence across multiple turns by retaining previous interactions.

In [30]:
print(f"\n--- Initializing Chat Session (Model: {chat_model_name}) ---")
try:
    model_with_persona = genai.GenerativeModel(
        chat_model_name,
        system_instruction=SYSTEM_PROMPT
    )
    chat_session = model_with_persona.start_chat(history=[])
    print("Chat session started successfully.")
except Exception as e:
    print(f"Error starting chat session: {e}")
    chat_session = None


--- Initializing Chat Session (Model: gemini-2.0-flash) ---
Chat session started successfully.


Testing final chat functionality

In [31]:
if chat_session:
    send_message_with_rag("Hi AuraMind, I'm feeling really stressed about a deadline again.")



User: Hi AuraMind, I'm feeling really stressed about a deadline again.
...
(Journal context included for this turn)

AuraMind:


> Hi there! It's completely understandable that you're feeling stressed about the deadline. Remember that feeling positive from finishing that report ahead of schedule? Maybe you can channel some of that energy into tackling this one, breaking it down into smaller, manageable steps. And don't forget those lunch breaks that help with focus! You've got this. How can I support you in managing this?



--- Evaluating Response ---
--- Sending to Evaluator --- 
You are an impartial evaluator AI. Your ONLY task is to evaluate the quality of an AI assistant's response based on the provided interaction details and criteria. You MUST output your evaluation ONLY in the specified JSON format. Do not include any introductory text, conversational filler, or anything else before or after the JSON object.


Interaction Details to Evaluate:
--------------------
User Query:
'''
Hi AuraMind, I'm feeling really stressed about a deadline again.
'''

Context Provided to AuraMind (Journal Entries or Search Results):
'''
--- Relevant Journal Entries Context ---
Feeling quite positive today! Managed to finish that report ahead of schedule.
---
Also remembered to take a proper lunch break today, which definitely helped my afternoon focus. Small wins! Need to keep doing that.
--- End of Context ---
'''

AuraMind's Response:
'''
Hi there! It's completely understandable that you're feeling stressed about th

In [32]:
if chat_session:
    send_message_with_rag("just a bit burned out, What's a quick breathing exercise I can do right now?")


User: just a bit burned out, What's a quick breathing exercise I can do right now?
...
(Journal context included for this turn)

AuraMind:


> I hear you. Burnout is tough. Since you're looking for a quick breather, how about trying the "4-7-8" technique? Breathe in quietly through your nose for 4 seconds, hold your breath for 7 seconds, and exhale slowly through your mouth for 8 seconds. It can really help calm the nervous system. Maybe it can give you a little of the calm you felt on those walks in nature you were mentioning. Would you like me to guide you through it?



--- Evaluating Response ---
--- Sending to Evaluator --- 
You are an impartial evaluator AI. Your ONLY task is to evaluate the quality of an AI assistant's response based on the provided interaction details and criteria. You MUST output your evaluation ONLY in the specified JSON format. Do not include any introductory text, conversational filler, or anything else before or after the JSON object.


Interaction Details to Evaluate:
--------------------
User Query:
'''
just a bit burned out, What's a quick breathing exercise I can do right now?
'''

Context Provided to AuraMind (Journal Entries or Search Results):
'''
--- Relevant Journal Entries Context ---
Also remembered to take a proper lunch break today, which definitely helped my afternoon focus. Small wins! Need to keep doing that.
---
It reminded me of the walks I used to take last year when I felt stressed. Nature really does seem to help calm my mind.
--- End of Context ---
'''

AuraMind's Response:
'''
I hear you. Burnout is t

In [33]:
if chat_session:
    send_message_with_rag("Can you remind me about that walk I took sometime back?")



User: Can you remind me about that walk I took sometime back?
...
(Journal context included for this turn)

AuraMind:


> Of course! I remember you mentioned taking a short walk near the Ram Ganga river one evening in Moradabad. You seemed to really enjoy it and it reminded you of how helpful nature is for calming your mind when stressed. Was there something specific about that walk you were hoping to remember?



--- Evaluating Response ---
--- Sending to Evaluator --- 
You are an impartial evaluator AI. Your ONLY task is to evaluate the quality of an AI assistant's response based on the provided interaction details and criteria. You MUST output your evaluation ONLY in the specified JSON format. Do not include any introductory text, conversational filler, or anything else before or after the JSON object.


Interaction Details to Evaluate:
--------------------
User Query:
'''
Can you remind me about that walk I took sometime back?
'''

Context Provided to AuraMind (Journal Entries or Search Results):
'''
--- Relevant Journal Entries Context ---
It reminded me of the walks I used to take last year when I felt stressed. Nature really does seem to help calm my mind.
---
The weather in Moradabad was lovely this evening. Took a short walk near the Ram Ganga river.
--- End of Context ---
'''

AuraMind's Response:
'''
Of course! I remember you mentioned taking a short walk near the Ram Ganga river one

In [34]:
if chat_session:
    send_message_with_rag("Thanks! Anything else I should keep in mind today?")



User: Thanks! Anything else I should keep in mind today?
...
(Journal context included for this turn)

AuraMind:


> You're welcome! Given that you've been prioritizing your well-being and experiencing the benefits, perhaps continue making time for those effective lunch breaks that boost your focus. Also, remember that feeling of accomplishment and positivity you felt after finishing the report ahead of schedule? You've shown you're capable of great things! Just keep acknowledging those small wins and building on them.



--- Evaluating Response ---
--- Sending to Evaluator --- 
You are an impartial evaluator AI. Your ONLY task is to evaluate the quality of an AI assistant's response based on the provided interaction details and criteria. You MUST output your evaluation ONLY in the specified JSON format. Do not include any introductory text, conversational filler, or anything else before or after the JSON object.


Interaction Details to Evaluate:
--------------------
User Query:
'''
Thanks! Anything else I should keep in mind today?
'''

Context Provided to AuraMind (Journal Entries or Search Results):
'''
--- Relevant Journal Entries Context ---
Also remembered to take a proper lunch break today, which definitely helped my afternoon focus. Small wins! Need to keep doing that.
---
Feeling quite positive today! Managed to finish that report ahead of schedule.
--- End of Context ---
'''

AuraMind's Response:
'''
You're welcome! Given that you've been prioritizing your well-being and experiencing the ben

## Conversation History

The entire chat history from the current chat session

In [35]:
print("\n--- Full Chat History ---")
if chat_session:
    for message in chat_session.history:
        display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
else:
    print("Chat session not available.")


--- Full Chat History ---


> **user**: --- Relevant Journal Entries Context ---
> Feeling quite positive today! Managed to finish that report ahead of schedule.
> ---
> Also remembered to take a proper lunch break today, which definitely helped my afternoon focus. Small wins! Need to keep doing that.
> --- End of Context ---
> 
> User Message: Hi AuraMind, I'm feeling really stressed about a deadline again.

> **model**: Hi there! It's completely understandable that you're feeling stressed about the deadline. Remember that feeling positive from finishing that report ahead of schedule? Maybe you can channel some of that energy into tackling this one, breaking it down into smaller, manageable steps. And don't forget those lunch breaks that help with focus! You've got this. How can I support you in managing this?


> **user**: --- Relevant Journal Entries Context ---
> Also remembered to take a proper lunch break today, which definitely helped my afternoon focus. Small wins! Need to keep doing that.
> ---
> It reminded me of the walks I used to take last year when I felt stressed. Nature really does seem to help calm my mind.
> --- End of Context ---
> 
> User Message: just a bit burned out, What's a quick breathing exercise I can do right now?

> **model**: I hear you. Burnout is tough. Since you're looking for a quick breather, how about trying the "4-7-8" technique? Breathe in quietly through your nose for 4 seconds, hold your breath for 7 seconds, and exhale slowly through your mouth for 8 seconds. It can really help calm the nervous system. Maybe it can give you a little of the calm you felt on those walks in nature you were mentioning. Would you like me to guide you through it?


> **user**: --- Relevant Journal Entries Context ---
> It reminded me of the walks I used to take last year when I felt stressed. Nature really does seem to help calm my mind.
> ---
> The weather in Moradabad was lovely this evening. Took a short walk near the Ram Ganga river.
> --- End of Context ---
> 
> User Message: Can you remind me about that walk I took sometime back?

> **model**: Of course! I remember you mentioned taking a short walk near the Ram Ganga river one evening in Moradabad. You seemed to really enjoy it and it reminded you of how helpful nature is for calming your mind when stressed. Was there something specific about that walk you were hoping to remember?


> **user**: --- Relevant Journal Entries Context ---
> Also remembered to take a proper lunch break today, which definitely helped my afternoon focus. Small wins! Need to keep doing that.
> ---
> Feeling quite positive today! Managed to finish that report ahead of schedule.
> --- End of Context ---
> 
> User Message: Thanks! Anything else I should keep in mind today?

> **model**: You're welcome! Given that you've been prioritizing your well-being and experiencing the benefits, perhaps continue making time for those effective lunch breaks that boost your focus. Also, remember that feeling of accomplishment and positivity you felt after finishing the report ahead of schedule? You've shown you're capable of great things! Just keep acknowledging those small wins and building on them.


## Contributions

Contributions by:¶
Shrey Sharma: [Kaggle](https://www.kaggle.com/shreysharma07)
Seoyeong oh(maia): [Kaggle](https://www.kaggle.com/maiaaaaaaaa)
Maryam Milad: [Kaggle](https://www.kaggle.com/maryammilad2001)
Sara jawad: [Kaggle](https://www.kaggle.com/sarajawadabadi)